In [1]:
import pandas as pd
import numpy as np

# ETL on Weather Data

This is farily experimental. Here I keep playing with Weather Data, iterating with visualizations and other parts until I have some clean data to work with. In the end only the Daily Data made much sense. Even so, I kept the rest of the data in case I might use it in other projects later on.

In [2]:
df1 = pd.read_csv('csvs/1746061.csv', parse_dates = [1], dtype={19: 'str', 30: 'str', 31: 'str', 33: 'str', 34: 'str', 41: 'str', 42: 'str', 43: 'str', 49: 'str', 51: 'str', 58: 'str', 69: 'str', 71: 'str', 75: 'str', 88: 'str', 89: 'str'})


In [ ]:
#df1 = pd.read_csv('~/Downloads/1746061.csv')

In [ ]:
df1.head()

## Separate Hourly, Daily, Monthly

The first step is to separate data, using just the columns required for that row type

In [ ]:
always = ["DATE", "REPORT_TYPE", "SOURCE"]
always = ["DATE"]

monthly = always+["AWND", "CDSD", "CLDD", "DSNW",  "MonthlyGreatestPrecip", "MonthlyGreatestPrecipDate", "MonthlyGreatestSnowDepth", "MonthlyGreatestSnowDepthDate", "MonthlyGreatestSnowfall", "MonthlyGreatestSnowfallDate", "MonthlyMaxSeaLevelPressureValue", "MonthlyMaxSeaLevelPressureValueDate", "MonthlyMaxSeaLevelPressureValueTime", "MonthlyMaximumTemperature", "MonthlyMeanTemperature", "MonthlyMinSeaLevelPressureValue", "MonthlyMinSeaLevelPressureValueDate", "MonthlyMinSeaLevelPressureValueTime", "MonthlyMinimumTemperature", "MonthlySeaLevelPressure", "MonthlyStationPressure", "MonthlyTotalLiquidPrecipitation", "MonthlyTotalSnowfall", "MonthlyWetBulb", "NormalsCoolingDegreeDay", "NormalsHeatingDegreeDay"]
daily = always+["DailyAverageDewPointTemperature", "DailyAverageDryBulbTemperature", "DailyAverageRelativeHumidity", "DailyAverageSeaLevelPressure", "DailyAverageStationPressure", "DailyAverageWetBulbTemperature", "DailyAverageWindSpeed", "DailyCoolingDegreeDays", "DailyDepartureFromNormalAverageTemperature", "DailyHeatingDegreeDays", "DailyMaximumDryBulbTemperature", "DailyMinimumDryBulbTemperature", "DailyPeakWindDirection", "DailyPeakWindSpeed", "DailyPrecipitation", "DailySnowDepth", "DailySnowfall", "DailySustainedWindDirection", "DailySustainedWindSpeed", "Sunrise", "Sunset"]
daily = always+["DailyAverageDryBulbTemperature", "DailyPeakWindSpeed", "DailyPrecipitation", "DailySnowDepth", "DailySnowfall", "Sunrise", "Sunset"]
hourly = always+["HourlyAltimeterSetting", "HourlyDewPointTemperature", "HourlyDryBulbTemperature", "HourlyPressureTendency", "HourlyRelativeHumidity", "HourlySeaLevelPressure", "HourlyStationPressure", "HourlyVisibility", "HourlyWetBulbTemperature", "HourlyWindDirection"]
hourly = always+["HourlyPrecipitation", "HourlyPresentWeatherType", "HourlySkyConditions", "HourlyWindSpeed"]
drop_cols = ["STATION", "ShortDurationEndDate010", "ShortDurationEndDate015", "ShortDurationEndDate020", "ShortDurationEndDate030", "ShortDurationEndDate045", "ShortDurationEndDate060", "ShortDurationEndDate080", "ShortDurationEndDate100", "ShortDurationEndDate120", "ShortDurationEndDate150", "ShortDurationEndDate180", "ShortDurationPrecipitationValue005", "ShortDurationPrecipitationValue010", "ShortDurationPrecipitationValue015", "ShortDurationPrecipitationValue020", "ShortDurationPrecipitationValue030", "ShortDurationPrecipitationValue045", "ShortDurationPrecipitationValue060", "ShortDurationPrecipitationValue080", "ShortDurationPrecipitationValue100", "ShortDurationPrecipitationValue120", "ShortDurationPrecipitationValue150", "ShortDurationPrecipitationValue180", "TStorms", "WindEquipmentChangeDate", "BackupDirection", "BackupDistance", "BackupDistanceUnit", "BackupElements", "BackupElevation", "BackupElevationUnit", "BackupEquipment", "BackupLatitude", "BackupLongitude", "BackupName"]
never = ["BackupDirection", "BackupDistance", "BackupDistanceUnit", "BackupElements", "BackupElevation", "BackupElevationUnit", "BackupEquipment", "BackupLatitude", "BackupLongitude", "BackupName"]
never = never + drop_cols


In [ ]:
df1["REPORT_TYPE"].unique()

In [ ]:
df1.drop(columns=never, inplace=True)
df1.shape

In [ ]:
df_daily = df1.loc[lambda df: df.REPORT_TYPE == "SOD  ", :][daily]
df_16 = df1.loc[lambda df: df.REPORT_TYPE == "FM-16", :][hourly]
df_15 = df1.loc[lambda df: df.REPORT_TYPE == "FM-15", :][hourly]
df_hourly = pd.concat([df_15, df_16])


In [ ]:
df_hourly.HourlyPrecipitation.unique()

## Hourly Data

Try to locate the weather types. Later on we might make a one-hot encoding out of it.

In [ ]:
#df_hourly["x1"] = df_hourly.HourlyPresentWeatherType.str.replace("[^A-Z ]", "", case=True, flags=0, regex=True).apply(lambda x: (list(set(x.split()))) if (isinstance(x, str) ) else np.NaN)
df_hourly["x1"] = df_hourly.HourlyPresentWeatherType.str.replace("[^A-Z ]", "", case=True, flags=0, regex=True).apply(lambda x: x.split() if (isinstance(x, str) ) else np.NaN)
df_hourly["x2"] = df_hourly.x1.apply(lambda x: x[-1] if (( isinstance(x, list) ) ) else np.NaN)

print("Weather Types are ", (df_hourly["x2"].unique()))

#df_hourly["x3"] = df_hourly.x1.apply(lambda x: x[1] if (( isinstance(x, list) and len(x) > 1 ) ) else np.NaN)
df_hourly = pd.get_dummies(df_hourly, prefix=None, columns = ["x2"])
df_hourly.rename(index=str, inplace=True, columns={"x2_BR": "BR", "x2_FG": "FG", "x2_FZ": "FZ", "x2_HZ": "HZ", "x2_RA": "RA", "x2_SN": "SN", "x2_UP": "UP", "x2_FU": "FU", "x2_FZRA": "FZRA"})

df_hourly.HourlyPrecipitation = pd.to_numeric(df_hourly.HourlyPrecipitation.str.replace("[A-Z]", "", case=False, flags=0, regex=True))
df_hourly = df_hourly.fillna(0)


#df_hourly["x4"].unique()



In [ ]:
df_hourly = df_hourly.drop(columns=["HourlyPresentWeatherType","HourlySkyConditions","x1" ])


In [ ]:
df_hourly.to_csv("csvs/hourly_weather_nyc.csv")

## Daily Data

There are a lot of gaps in the daily data, and just some strange special characters. One cleanup routine helps across all of the daily data rollups.

In [ ]:
def cleanup(x):
    #print("A1",x.unique())
    x  = x.str.replace("[A-Z]", "", case=False, flags=0, regex=True)
    #print("A2",x.unique())
    x  = x.str.replace("*", "", case=False, flags=0, regex=False)
    #print("A3",x.unique())
    x  = pd.to_numeric(x)
     #x.astype(int)
    #print("B",x.unique())
  
    return x
#df_daily[df_daily['DailySnowDepth'].str.match("")]

In [ ]:

df_daily['DailyAverageDryBulbTemperature2'] = cleanup(df_daily.DailyAverageDryBulbTemperature)
df_daily['DailyPeakWindSpeed2'] = cleanup(df_daily.DailyPeakWindSpeed)
df_daily['DailyPrecipitation2'] = cleanup(df_daily.DailyPrecipitation)
df_daily['DailySnowDepth2'] = cleanup(df_daily.DailySnowDepth)
df_daily['DailySnowfall2'] = cleanup(df_daily.DailySnowfall)
df_daily = df_daily.fillna(0)

df_daily.head()


## Wind Speed

Wind Speed is broken in many places. 2,237 miles per hour? We'll need to fix that later.

In [ ]:
df_daily['DailyPeakWindSpeed2'].unique()

In [ ]:
df_daily = df_daily.drop(columns=["DailyAverageDryBulbTemperature","DailyPeakWindSpeed","DailyPrecipitation","DailySnowDepth","DailySnowfall" ])

df_daily.rename(index=str, inplace=True, columns={"DailyAverageDryBulbTemperature2":"DailyAverageDryBulbTemperature", "DailyPeakWindSpeed2":"DailyPeakWindSpeed", "DailyPrecipitation2":"DailyPrecipitation", "DailySnowDepth2":"DailySnowDepth", "DailySnowfall2":"DailySnowfall"})


print("Cleaned up Daily")
df_daily.to_csv("csvs/daily_weather_nyc.csv")
df_daily.head()
